## OR - Assignment 1

##### Author: Joaquim Marset Alsina

### Imports

In [1]:
import os
import matplotlib.pyplot as plt
import random
from dataset import *
from generator import ImageGenerator
from train import Trainer
from load_utils import *
from segmentation_utils import extract_segmentations, filter_segmentations_train
from model_utils import mobile_name, resnet_name, inception_name

### Constants

In [2]:
root_path = "."
train_val_path = os.path.join(root_path, 'VOCtrainval_06-Nov-2007', 'VOCdevkit', 'VOC2007')

segmentations_path = os.path.join(train_val_path, "SegmentationObject")
annotations_path = os.path.join(train_val_path, "Annotations")
images_path = os.path.join(train_val_path, "JPEGImages")

pickle_path = os.path.join(root_path, "segmentations.pkl")

results_path = os.path.join(root_path, 'results')
results_mobile_path = os.path.join(results_path, mobile_name)
results_resnet_path = os.path.join(results_path, resnet_name)
results_inception_path = os.path.join(results_path, inception_name)

In [3]:
seed = 1412
image_size = 224
num_classes = 20
val_percentage = 0.2
prob_augment = 0.5
batch_size = 32
num_epochs = 5

### Create required folders

In [4]:
if not exists_path(results_path):
    create_results_folder(results_path)
    create_results_folder(results_mobile_path)
    create_results_folder(results_resnet_path)
    create_results_folder(results_inception_path)

### Extract segmentation objects

In [5]:
if not exists_path(pickle_path):
    segmentation_objects = extract_segmentations(images_path, segmentations_path, annotations_path, image_size)
    save_segmentations(pickle_path, segmentation_objects)
else:
    segmentation_objects = load_segmentations_pickle(pickle_path)

### Create train and validation sets

In [6]:
images_names = [image_file[:-4] for image_file in os.listdir(images_path)]
random.Random(seed).shuffle(images_names)

In [7]:
train_names, val_names = create_train_val_split(images_names, val_percentage, seed)

# Among the segmented objects, pick only from the current training split
train_segmentations = filter_segmentations_train(segmentation_objects, train_names)
del segmentation_objects

### Experiments

In [8]:
# Unchanged during all experiments
val_dataset = Dataset(images_path, annotations_path, val_names, image_size, seed)
val_generator = ImageGenerator(batch_size, val_dataset, seed)

In [9]:
def run_experiment(model_name, train_gen, fine_tune, experiment_name):
    file_suffix, plot_title = experiment_name
    trainer = Trainer(results_path, "", model_name, image_size, num_classes, fine_tune)
    trainer.train(num_epochs, train_gen, val_generator, file_suffix, plot_title)

#### MobileNetv2 - No augmentation

In [10]:
"""
augmentation_mode = AugmentationMode.NoAugmentation

train_dataset = TrainDataset(images_path, annotations_path, train_names, image_size, train_segmentations, 
    augmentation_mode, overlap, num_to_place, prob_augment, seed)

train_generator = ImageGenerator(batch_size, train_dataset, seed)

experiment_name_file = 'no_augmentation'
experiment_name_title = 'No augmentation'

run_experiment(mobile_name, train_generator, True, (experiment_name_file, experiment_name_title))
"""

"\naugmentation_mode = AugmentationMode.NoAugmentation\n\ntrain_dataset = TrainDataset(images_path, annotations_path, train_names, image_size, train_segmentations, \n    augmentation_mode, overlap, num_to_place, prob_augment, seed)\n\ntrain_generator = ImageGenerator(batch_size, train_dataset, seed)\n\nexperiment_name_file = 'no_augmentation'\nexperiment_name_title = 'No augmentation'\n\nrun_experiment(mobile_name, train_generator, True, (experiment_name_file, experiment_name_title))\n"

#### MobileNetv2 - Augmentation without overlap

In [ ]:
augmentation_mode = AugmentationMode.Augmentation
num_objects_to_place = 3

train_dataset = TrainDataset(images_path, annotations_path, train_names, image_size, train_segmentations, 
    augmentation_mode, num_objects_to_place, prob_augment, seed)

train_generator = ImageGenerator(batch_size, train_dataset, seed)

experiment_name_file = 'augmentation'
experiment_name_title = 'Augmentation'

run_experiment(mobile_name, train_generator, True, (experiment_name_file, experiment_name_title))

#### MobileNetv2 - Augmentation with overlap

#### MobileNetv2 - Augmentation with overlap and random transformations

#### MobileNetv2 - Augmentation with overlap and equal proportions of labels